# Data Wrangling
The goal of this notebook is to wrangle the Ethereum-USD Coin pricing data into a workable format to then feed to machine learning algorithms further down the road.

In [116]:
import pandas as pd
from datetime import datetime

### Load data into pandas and some preliminary data analasys

In [117]:
df = pd.read_json('../data/raw/ETH-USD.json')
df.reset_index(drop=False, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   1000 non-null   int64  
 1   0       1000 non-null   int64  
 2   1       1000 non-null   float64
 3   2       1000 non-null   float64
 4   3       1000 non-null   float64
 5   4       1000 non-null   float64
 6   5       1000 non-null   float64
 7   6       1000 non-null   int64  
 8   7       1000 non-null   float64
 9   8       1000 non-null   int64  
 10  9       1000 non-null   float64
 11  10      1000 non-null   float64
 12  11      1000 non-null   int64  
dtypes: float64(8), int64(5)
memory usage: 101.7 KB


In [118]:
df.head(3)

,index,0,1,2,3,4,5,6,7,8,9,10,11
0,0,1689302600000,1600.0,1600.0,1600.0,1600.0,0.0000,1689302600999,0.00,0,0.0,0.0,0
1,1,1689302601000,1600.0,1600.0,1600.0,1600.0,0.0000,1689302601999,0.00,0,0.0,0.0,0
2,2,1689302602000,1600.0,1600.0,1600.0,1600.0,0.0962,1689302602999,153.92,1,0.0,0.0,0


Now we add headers to the columns

In [119]:
df = df.iloc[:, 1:]
df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_trades', 'base', 'quote', 'unused']
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_trades,base,quote,unused
0,1689302600000,1600.00,1600.00,1600.00,1600.00,0.0000,1689302600999,0.000000,0,0.0000,0.000000,0
1,1689302601000,1600.00,1600.00,1600.00,1600.00,0.0000,1689302601999,0.000000,0,0.0000,0.000000,0
2,1689302602000,1600.00,1600.00,1600.00,1600.00,0.0962,1689302602999,153.920000,1,0.0000,0.000000,0
3,1689302603000,1600.05,1600.05,1600.05,1600.05,0.0628,1689302603999,100.483140,3,0.0628,100.483140,0
4,1689302604000,1600.05,1616.25,1600.05,1616.25,0.8331,1689302604999,1336.929089,17,0.8331,1336.929089,0


Remove Unnessesary Columns

In [120]:
df = df.iloc[:, :-3]
df.drop(columns='close_time', inplace=True)
df.head(-5)

,open_time,open,high,low,close,volume,quote_asset_volume,number_trades
0,1689302600000,1600.00,1600.00,1600.00,1600.00,0.0000,0.000000,0
1,1689302601000,1600.00,1600.00,1600.00,1600.00,0.0000,0.000000,0
2,1689302602000,1600.00,1600.00,1600.00,1600.00,0.0962,153.920000,1
3,1689302603000,1600.05,1600.05,1600.05,1600.05,0.0628,100.483140,3
4,1689302604000,1600.05,1616.25,1600.05,1616.25,0.8331,1336.929089,17
...,...,...,...,...,...,...,...,...
990,1689303590000,1595.45,1595.45,1595.45,1595.45,0.0000,0.000000,0
991,1689303591000,1595.45,1595.45,1595.45,1595.45,0.0000,0.000000,0
992,1689303592000,1595.45,1595.45,1595.45,1595.45,0.0000,0.000000,0
993,1689303593000,1595.45,1595.45,1595.45,1595.45,0.0000,0.000000,0


Convert ```open_time```  timestamp to a datetime object

In [121]:
# kept getting a weird error trying to convert timestamps, had to remove the milliseconds from the timestamp.
# https://stackoverflow.com/questions/31548132/python-datetime-fromtimestamp-yielding-valueerror-year-out-of-range

df['open_time'] = [datetime.fromtimestamp(x / 1000) for x in df['open_time']]
df.dtypes

open_time             datetime64[ns]
open                         float64
high                         float64
low                          float64
close                        float64
volume                       float64
quote_asset_volume           float64
number_trades                  int64
dtype: object

Now set the date as the index.

In [122]:
df.set_index('open_time', inplace=True)
df.head()

,open,high,low,close,volume,quote_asset_volume,number_trades
open_time,,,,,,,
2023-07-13 19:43:20,1600.00,1600.00,1600.00,1600.00,0.0000,0.000000,0
2023-07-13 19:43:21,1600.00,1600.00,1600.00,1600.00,0.0000,0.000000,0
2023-07-13 19:43:22,1600.00,1600.00,1600.00,1600.00,0.0962,153.920000,1
2023-07-13 19:43:23,1600.05,1600.05,1600.05,1600.05,0.0628,100.483140,3
2023-07-13 19:43:24,1600.05,1616.25,1600.05,1616.25,0.8331,1336.929089,17


Now write cleaned data to the processed directory.

In [123]:
df.to_csv('../data/processed/ETH-USD.csv')